In [1]:
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from webdriver_manager.chrome import ChromeDriverManager
import time 
from config import dbcon
from sqlalchemy import create_engine
from sqlalchemy import Column, Integer, Float, String, DateTime, BIGINT
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import create_engine
from sqlalchemy import UniqueConstraint, ForeignKey
from sqlalchemy.orm import relationship
import numpy as np
import psycopg2

In [2]:
df = pd.read_csv('Data/DB structure/IOWA_Product_Item.csv', encoding="ISO-8859-1")
dfProductCategory = pd.read_csv('Data/DB structure/IOWA_Product_Category.csv', encoding="ISO-8859-1")
dfStoreData = pd.read_csv('Data/DB structure/IOWA_store_data.csv', encoding="ISO-8859-1")

In [ ]:
url = 'https://shop.iowaabd.com/'
driver = webdriver.Chrome(ChromeDriverManager().install())
driver.get(url)
driver.implicitly_wait(3)
df['image_url'] = ''
for index, row in df.iterrows():       
    driver.find_element_by_name('Search').send_keys(row['item_description'])
    driver.find_element_by_xpath('//*[@id="searchform1"]/div/input[2]').click()    
    img = driver.find_elements_by_class_name('productimage')
    if(img):
        img_url = img[0].get_attribute("src")
        df.at[index,'image_url'] = img_url
    else:
        img_url = 'https://shop.iowaabd.com/images/product-small/image-not-available2.jpg'
    print(str(index) + ' ' +img_url)
    time.sleep(0.5)
    driver.find_element_by_name('Search').clear()

driver.quit() 
df.to_csv('Data/DB structure/IOWA_Product_Item_Image.csv',index=False)


In [3]:
df = pd.read_csv('Data/DB structure/IOWA_Product_Item_Image.csv', encoding="ISO-8859-1")
dfProduct_Items = df.rename(columns={'state_bottle_cost': 'cost', 'state_bottle_retail': 'price', 'bottle_volume_ml': 'volume'})
dfProduct_Items = dfProduct_Items.drop_duplicates(subset='item_number', keep="first")
dfProduct_Items['category'] = dfProduct_Items['category'].fillna(-1)
dfProduct_Items['category'] = dfProduct_Items['category'].astype(int)

dfProduct_Items.head()

,item_number,item_description,category,cost,price,volume,image_url
0,76036,Midnight Moon Apple Pie,1091300,11.50,17.25,750,https://shop.iowaabd.com/images/product-small/...
1,86310,La Prima Triple Sec,1081500,3.00,4.50,1000,https://shop.iowaabd.com/images/product-small/...
2,26613,Foundry Corn Whiskey,1011700,40.00,60.00,750,https://shop.iowaabd.com/images/product-small/...
3,33717,Paramount Sloe Gin,1041300,5.42,8.13,1000,https://shop.iowaabd.com/images/product-small/...
4,27102,Templeton 4YR Rye,1011600,18.09,27.14,750,https://shop.iowaabd.com/images/product-small/...


In [4]:
dfProductCategory = dfProductCategory.dropna()
dfProductCategory['category'] = dfProductCategory['category'].astype(int)
dfProductCategory.loc[-1] = [-1,'No Category']
dfProductCategory = dfProductCategory.drop_duplicates(subset='category', keep="first")
dfProductCategory.head()

,category,category_name
0,1012300,Single Malt Scotch
1,1011600,Straight Rye Whiskies
2,1901200,Special Order Items
3,1081500,Triple Sec
4,1062100,Gold Rum


In [5]:
dfStoreData = dfStoreData.dropna()
dfStoreData['store_number'] = dfStoreData['store_number'].astype(int)
dfStoreData = dfStoreData.drop_duplicates(subset='store_number', keep="first")
dfStoreData.head()

,store_number,store_name,store_location,address,city
0,2652,Hy-Vee DrugStore / Mason City,POINT (-93.213856 43.148257),875 4th St SW,Mason City
1,5077,Wal-Mart 3150 / Council Bluffs,POINT (-95.867459 41.27737),1800 N 16th St,Council Bluffs
2,2666,Hy-Vee Food Store #2 / State Ankeny,POINT (-93.621824 41.705188),2510 SW State St,Ankeny
3,2571,Hy-Vee Food Store #2 / Waterloo,POINT (-92.337583 42.530476),2181 Logan Ave,Waterloo
4,5001,The Music Station / Independence,POINT (-91.901687 42.468633),709 First St W,Independence


In [ ]:
print(dbcon)

In [6]:
Base = declarative_base()
dbcon = f'{dbcon}'
Engine = create_engine(dbcon)

class Product_Catogory(Base):
    __tablename__ = 'product_catogory'
    __table_args__ = (UniqueConstraint('category'),)    
    category = Column(Integer, primary_key=True)
    category_name = Column(String, nullable=False)

class Product_Items(Base):
    __tablename__ = 'product_items'
    __table_args__ = (UniqueConstraint('item_number'),)    
    item_number = Column(Integer, primary_key=True)
    item_description = Column(String, nullable=False)
    category = Column(Integer, ForeignKey('product_catogory.category'), nullable=False) 
    cost = Column(Float, nullable=False)
    price = Column(Float, nullable=False)
    volume = Column(Float, nullable=False)
    image_url = Column(String, nullable=False)
    
class Store_Details (Base):
    __tablename__ = 'store_details'
    __table_args__ = (UniqueConstraint('store_number'),)    
    store_number = Column(Integer, primary_key=True)
    store_name = Column(String, nullable=False)
    store_location = Column(String, nullable=False)
    address = Column(String, nullable=False)
    city = Column(String, nullable=False)
    
Base.metadata.drop_all(Engine)
Base.metadata.create_all(Engine)

with Engine.connect() as dbcon:
    with dbcon.begin() as dbtrans:
        dbcon.execute(Product_Catogory.__table__.insert(), dfProductCategory.to_dict(orient='records'))
        dbcon.execute(Product_Items.__table__.insert(), dfProduct_Items.to_dict(orient='records'))
        dbcon.execute(Store_Details.__table__.insert(), dfStoreData.to_dict(orient='records'))
        dbtrans.commit() 